In [1]:
# pip install langchain-openai
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_core.messages.human import HumanMessage
from langchain_core.messages.system import SystemMessage
from langchain_core.runnables.config import RunnableConfig

from dotenv import load_dotenv
load_dotenv() #loads `OPENAI_API_KEY`

True

In [2]:
prompt = """Write a weather report for a major city
    in ten words or less.
    Do not include any additional explanation.
"""

guided_prompt = prompt + """
Return the result as JSON as follows:
{ "city": "<CITY_NAME>",
"report": "<BRIEF_REPORT>" }
"""

In [3]:
chat = ChatOpenAI(
    #base_url="http://localhost:1234/v1",
    temperature=0.7,
    max_tokens=500,
    model='gpt-4-1106-preview'
)

In [4]:
def baseline():
    print("baseline:")
    print(chat.invoke(prompt).content)

In [5]:
def with_guided_prompt():
    print("1. Ask nicely")
    print(chat.invoke(guided_prompt).content)

In [6]:
def with_pydantic_output_formatter():
    print("2. Pydantic OutputParser")
    from langchain_core.output_parsers.pydantic import PydanticOutputParser
    from langchain_core.utils.pydantic import BaseModel, Field

    class WeatherReport(BaseModel):
        city: str = Field(description="City name")
        report: str = Field(description="Brief weather report")

    parser = PydanticOutputParser(pydantic_object=WeatherReport)
    #print(f"Parser instructions: {parser.get_format_instructions()}")

    runnable_prompt = ChatPromptTemplate.from_messages(
        [
            SystemMessage(content=parser.get_format_instructions()),
            HumanMessage(content=prompt)
        ]
    )
    chain = runnable_prompt | chat | parser
    py_obj = chain.invoke({})
    # print(py_obj.city, py_obj.report)
    print(py_obj)

    #To be extra robust about fixing JSON errors you can add
    #from langchain.output_parsers import OutputFixingParser
    #robust_parser = OutputFixingParser.from_llm(
    #    parser=parser,
    #    llm=chat
    #)
    # this will re-prompt to get a conforming output format

In [7]:
baseline()

baseline:
New York: Sunny skies, mild breeze, high of 68°F today.


In [8]:
with_guided_prompt()

1. Ask nicely
```json
{
  "city": "New York",
  "report": "Sunny, high of 76, clear skies."
}
```


In [9]:
with_pydantic_output_formatter()

2. Pydantic OutputParser
city='New York' report='Sunny skies, mild breeze, perfect spring day.'
